In [79]:
import nltk
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelBinarizer
import sklearn
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
import numpy as np
import pandas as pd
import re

In [80]:
# baca file train dan test
data_train = pd.read_csv("dataset.tsv", delimiter="\t", header=None)
data_test = pd.read_csv("data_test.tsv", delimiter="\t", header=None)
most_word = []
with open("most_word.txt", "r") as file:
    for line in file.readlines():
        temp = line.replace('\x00', '')
        most_word.append(temp.rstrip('\n'))

In [81]:
# menghapus '' kosong di dalam array dengan membuat array baru
most_w =  []
for i in most_word:
    if i != '':
        most_w.append(i)
most_w

['ÿþsebagai',
 'saya',
 'nya',
 'bahwa',
 'dia',
 'adalah',
 'untuk',
 'pada',
 'adalah',
 'dengan',
 'mereka',
 'menjadi',
 'di',
 'satu',
 'memiliki',
 'ini',
 'dari',
 'oleh',
 'hot',
 'kata',
 'tapi',
 'apa',
 'beberapa',
 'adalah',
 'itu',
 'anda',
 'atau',
 'memiliki',
 'itu',
 'dari',
 'untuk',
 'dan',
 'sebuah',
 'di',
 'kami',
 'bisa',
 'out',
 'lainnya',
 'yang',
 'yang',
 'melakukan',
 'mereka',
 'waktu',
 'jika',
 'akan',
 'bagaimana',
 'kata',
 'an',
 'masing-masing',
 'memberitahu',
 'tidak',
 'Kumpulan',
 'tiga',
 'ingin',
 'udara',
 'baik',
 'juga',
 'bermain',
 'kecil',
 'end',
 'menempatkan',
 'rumah',
 'baca',
 'tangan',
 'pelabuhan',
 'besar',
 'mantra',
 'tambahkan',
 'bahkan',
 'tanah',
 'di sini',
 'harus',
 'besar',
 'tinggi',
 'seperti',
 'ikuti',
 'tindakan',
 'mengapa',
 'bertanya',
 'laki-laki',
 'perubahan',
 'pergi',
 'cahaya',
 'jenis',
 'off',
 'perlu',
 'rumah',
 'gambar',
 'coba',
 'kami',
 'lagi',
 'hewan',
 'titik',
 'ibu',
 'dunia',
 'dekat',
 'memb

In [82]:
# fungsi untuk konversi label NE
def convert_label(raw_ne):
    new_label = 0
    if (raw_ne=='B-PER'): new_label = 1
    if (raw_ne=='I-PER'): new_label = 2
    if (raw_ne=='B-ORG'): new_label = 3
    if (raw_ne=='I-ORG'): new_label = 4
    if (raw_ne=='B-LOC'): new_label = 5
    if (raw_ne=='I-LOC'): new_label = 6
    return new_label

In [83]:
# inisialisasi kode token dan postag
token_dict = {}
postag_dict = {}
counter_token = 0
counter_postag = 0

In [84]:
# baca data token, postag, dan label NE
train_sents = []
test_sents = []
sent = []

In [85]:
# data train
for row in data_train.iterrows():
    word = row[1][0]
    tag = row[1][1]
    label = row[1][2]
    if not row[1].isnull()[2]:
        t = (word, tag, convert_label(label))
        if word.lower() not in token_dict.keys():
            token_dict[word.lower()] = counter_token
            counter_token += 1
        if tag not in postag_dict.keys():
            postag_dict[tag] = counter_postag
            counter_postag += 1
        sent.append(t)
    else:
        if len(sent) == 0:
            continue
        print('Train sent:', sent)
        train_sents.append(sent)
        sent = []

Train sent: [('SEA', 'NN', 3), ('GAMES', 'NN', 4), ('2019', 'CDP', 4), ('merupakan', 'VBT', 0), ('sebuah', 'NN', 0), ('Pesta', 'NN', 3), ('Olahraga', 'NN', 4), ('Asia', 'NNP', 4), ('Tenggara', 'NN', 4), ('2019', 'NN', 4), ('yang', 'SC', 0), ('akan', 'MD', 0), ('diselenggarakan', 'VBT', 0), ('di', 'IN', 0), ('Manila', 'NNP', 5)]
Train sent: [('ibu', 'NNP', 0), ('kota', 'NN', 0), ('dari', 'IN', 0), ('negara', 'NN', 0), ('Filipina', 'NN', 5)]
Train sent: [('Pesta', 'NN', 3), ('Olahraga', 'NN', 4), ('Asia', 'NNP', 4), ('Tenggara', 'NN', 4), ('kali', 'NN', 0), ('ini', 'DT', 0), ('akan', 'MD', 0), ('menjadi', 'VBT', 0), ('Pesta', 'NN', 3), ('Olahraga', 'NN', 4), ('Asia', 'NNP', 4), ('Tenggara', 'NN', 4), ('yang', 'SC', 0), ('ke', 'IN', 0)]
Train sent: [('30', 'CDP', 0), ('(', 'OP', 0), ('tiga', 'CDP', 0), ('puluh)', 'CDP', 0)]
Train sent: [('Pesta', 'NN', 3), ('Olahraga', 'NN', 4), ('Asia', 'NNP', 4), ('Tenggara', 'NN', 4), ('ini', 'DT', 0), ('akan', 'MD', 0), ('diselenggarakan', 'VBT', 0), 

In [86]:
for row in data_test.iterrows():
    word = row[1][0]
    tag = row[1][1]
    label = row[1][2]
    if not row[1].isnull()[2]:
        t = (word, tag, convert_label(label))
        sent.append(t)
    else:
        if len(sent) == 0:
            continue
        print('Test sent:', sent)
        test_sents.append(sent)
        sent = []

Test sent: [('Pada', 'IN', 0), ('pemberitaan', 'NN', 0), ('sebelumnya', 'NNG', 0)]
Test sent: [('September', 'NN', 0)]
Test sent: [('Datu', 'NNP', 1)]
Test sent: [('Ramos', 'NNP', 2), ('sudah', 'MD', 0), ('mencuatkan', 'VBT', 0), ('isu', 'NNP', 0), ('terkait', 'VBT', 0), ('sertifikasi', 'NN', 0), ('halal', 'NN', 0), ('katering', 'NN', 0), ('untuk', 'IN', 0), ('para', 'DT', 0), ('tamu', 'NN', 2), ('muslim', 'NN', 4), ('di', 'IN', 0), ('SEA', 'NN', 4), ('Games', 'NN', 4), ('2019', 'CDP', 4)]
Test sent: [('Dia', 'PRP', 1), ('juga', 'RB', 0), ('sempat', 'RB', 0), ('mempertanyakan', 'VBT', 0), ('ketersediaan', 'NN', 0), ('tempat', 'NN', 0)]
Test sent: [('tempat', 'NN', 0), ('salat', 'NN', 0), ('dengan', 'IN', 0), ('petunjuk', 'NN', 0), ('arah', 'NN', 0), ('kiblat', 'NN', 0), ('yang', 'SC', 0), ('sesuai', 'RB', 0)]
Test sent: [('Pada', 'IN', 0), ('Minggu', 'NN', 0), ('(', 'OP', 0), ('24/11)', '11)', 0)]
Test sent: [('Chef', 'NNP', 1), ('de', 'NNP', 2), ('Mission', 'NNP', 2), ('Singapura', 'N

In [87]:
token_dict['unk'] = 9999
postag_dict['unk'] = 9999

In [88]:
def word2features(sent, i):  
    word = sent[i][0]
    postag = sent[i][1]
    
    prev_word = sent[i-1][0] if i >= 1 else 'unk'
    prev_tag = sent[i-1][1] if i >= 1 else 'unk'
    
    next_word = sent[i+1][0] if i < len(sent)-1 else 'unk'
    next_tag = sent[i+1][1] if i < len(sent)-1 else 'unk'
    
    if word.lower() not in token_dict.keys(): 
        word = 'unk'
    if postag not in postag_dict.keys():
        postag = 'unk'
        
    if prev_word.lower() not in token_dict.keys():
        prev_word = 'unk'
        
    if prev_tag not in postag_dict.keys():
        prev_tag = 'unk'
        
    if next_word.lower() not in token_dict.keys():
        next_word = 'unk'
    
    if next_tag not in postag_dict.keys():
        next_tag = 'unk'
        
    features = [
        token_dict[word.lower()], # fitur kata dalam bentuk lowercase
        word.isupper(), # fitur apakah karakter pertama token merupakan huruf kapital
        word.istitle(), # fitur apakah token merupakan title
        word.isdigit(), # fitur apakah token merupakan digit
        postag_dict[postag], # fitur kode postag token
        token_dict[prev_word.lower()], # fitur kata sebelum
        postag_dict[prev_tag], # fitur postag sebelum
        token_dict[next_word.lower()], # fitur kata setelah
        postag_dict[next_tag], # fitur postag setelah,
        word.lower() in most_w # fitur kata populer
    ]
                
    return features

In [89]:
def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent] 

In [90]:
print('postag dictonary')
print(postag_dict)
print('token dictonary')
print(token_dict)

postag dictonary
{'NN': 0, 'CDP': 1, 'VBT': 2, 'NNP': 3, 'SC': 4, 'MD': 5, 'IN': 6, 'DT': 7, 'OP': 8, '-': 9, 'CC': 10, 'CDO': 11, 'JJ': 12, 'CDI': 13, 'RB': 14, 'VBI': 15, 'CDC': 16, 'WP': 17, 'CP': 18, 'NEG': 19, 'PRP': 20, 'NNG': 21, '11': 22, 'RP': 23, '.': 24, 'FW': 25, ',': 26, 'unk': 9999}
token dictonary
{'sea': 0, 'games': 1, '2019': 2, 'merupakan': 3, 'sebuah': 4, 'pesta': 5, 'olahraga': 6, 'asia': 7, 'tenggara': 8, 'yang': 9, 'akan': 10, 'diselenggarakan': 11, 'di': 12, 'manila': 13, 'ibu': 14, 'kota': 15, 'dari': 16, 'negara': 17, 'filipina': 18, 'kali': 19, 'ini': 20, 'menjadi': 21, 'ke': 22, '30': 23, '(': 24, 'tiga': 25, 'puluh)': 26, 'quirino': 27, 'grandstand': 28, 'rizal': 29, 'park': 30, 'empat': 31, 'dalam': 32, 'sejarah': 33, 'menggelar': 34, '-': 35, 'dan': 36, 'kedua': 37, 'setelah': 38, '14': 39, 'tahun': 40, 'telah': 41, 'antara': 42, 'lain': 43, 'pada': 44, '1981': 45, '1991': 46, '2005': 47, 'sedangkan': 48, 'beberapa': 49, 'pertandingan': 50, 'metro': 51, 'l

In [91]:
X_train = []
y_train = []
for s in train_sents:
    for i in range(len(s)):
        X_train.append(word2features(s,i))
        y_train.append(s[i][2])

In [92]:
X_test = []
y_test = []
for s in test_sents:
    for i in range(len(s)):
        X_test.append(word2features(s,i))
        y_test.append(s[i][2])

In [93]:
X_train = np.array(X_train)
y_train = np.array(y_train)

X_test = np.array(X_test)
y_test = np.array(y_test)

In [94]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [95]:
print('xtest')
print(X_test[0])

print(clf.predict(X_test[0].reshape(1,-1)))
print('Actual', y_test[0])

xtest
[  44    0    1    0    6 9999 9999 9999    0    1]
[3]
Actual 0


In [96]:
print('hasil klasifikasi data test:')
res = clf.predict(X_test)
print(res)

hasil klasifikasi data test:
[3 2 2 0 5 1 4 2 0 2 1 0 2 1 2 2 2 1 1 4 4 1 1 4 2 1 2 3 2 0 2 2 1 0 2 3 2
 2 2 0 2 1 2 0 2 1 1 1 4 0 0 4 4 2 2 2 1 1 1 4 2 2 1 0 2 1 1 4 4 2 2 1 1 0
 4 1 0 2 1 1 2 4 2 1 4 2 1 4 2 1 2 0 4 2 1 1 4 2 2 3 2 1 0 2 1 1 1 1 1 4 4
 1 1 4 2 0 2 1 1 2 4 0 2 2 1 1 2 1 1 2 4 2 0 2 1 2 4 1 4 2 1 0 2 1 4 2 0 2
 1 1 4 1 4 2 0 0 2 2 1 1 1 4 2 2 1 1 2 2 4 2 0 2 2 1 0 2 1 4 0 1 1 4 2 1 2
 4 3 2 1 1 4 1 2 2 2 2 1 0 4 2 1 1 4 2 0 1 0 2 1 4 0 2 3 2 2 1 1 4 0 0 2 0
 1 0 2 1 2 4 2 1 1 2 4 3 2 1 2 1 4 4 1 2 4 3 2 0 1 1 4 2 1 2 4 2 1 2 2 4 0
 2 1 0 2 0 2 1 0 2 0 2 0 2 1 4 0 1 0 2 1 0 2 2 1 4 2 0 2 1 2 1 1 2 4 2 0 2
 1 0 2 0 2 2 3 2 0 2 2 1 1 4 2 1 4 2 3 2 0 1 1 4 1 0 2 1 1 4 0 1 1 0 2 2 1
 0 2 2 4 2 1 4 2 1 4 3 2 1 1 4 1 4 2 2 3 2 0 2 0 2 3 2 0 2 2 2 1 0 2 2 2 1
 1 4 2 0 2 0 2 0 2 1 0 4 3 2 2 2 1 1 4 2 0 2 2 2 1 0 2 1 4 0 1 0 2 1 4 2 0
 2 2 0 2]


In [97]:
correct = 0
for x, y in zip(res, y_test):
    if x == y:
        correct += 1
print('Akurasi:', correct/len(res))

Akurasi: 0.20681265206812652
